In [119]:
import chromadb
import langchain
#import pypdf
import sentence_transformers
#import tiktoken
import openai
from chromadb.utils import embedding_functions

In [120]:
#from langchain.document_loaders import PyPDFLoader

In [121]:
from langchain.document_loaders import TextLoader

In [122]:
#from langchain.document_loaders import Docx2txtLoader

In [123]:
#from langchain.text_splitter import CharacterTextSplitter

In [124]:
from langchain.embeddings import HuggingFaceHubEmbeddings

In [125]:
from langchain.vectorstores import Chroma

In [126]:
from huggingface_hub import notebook_login

In [127]:
#import torch
#import transformers

In [128]:
#from transformers import AutoTokenizer, AutoModelForCausalLM

In [129]:
from transformers import pipeline

In [130]:
from langchain import HuggingFacePipeline

In [131]:
from langchain.chains import ConversationalRetrievalChain

In [132]:
from langchain.memory import ConversationBufferMemory

In [133]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

In [134]:
import os
import sys

In [135]:
!mkdir Procom_Collection

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory ‘Procom_Collection’: File exists


In [136]:
import docx

def read_docx_as_single_page(docx_path):
    doc = docx.Document(docx_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

In [137]:
from PyPDF2 import PdfReader

def read_pdf_as_single_page(pdf_path):
    with open(pdf_path, "rb") as f:
        pdf_reader = PdfReader(f)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            # Extract text from the current page and append it to the string
            text += pdf_reader.pages[page_num].extract_text()
        return text


In [138]:
document = []
for file in os.listdir("Procom_test"):
    if file.endswith(".pdf"):
        pdf_path = "./Procom_test/" + file
        document.append(read_pdf_as_single_page(pdf_path))  # Append instead of extend
    elif file.endswith(".docx") or file.endswith(".doc"):
        doc_path = "./Procom_test/" + file
        document.append(read_docx_as_single_page(doc_path))  # Append instead of extend
    elif file.endswith(".txt"):
        txt_path = "./Procom_test/" + file
        loader = TextLoader(txt_path)
        document.append(loader.load())  # Append instead of extend


In [ ]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

In [ ]:
chroma_client = chromadb.PersistentClient("./Procom_KnowledgeBase")

In [ ]:
# import chromadb.utils.embedding_functions as embedding_functions
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#             api_key="YOUR API KEY",
#             model_name="text-embedding-ada-002"
#             )


In [ ]:
client = chroma_client.get_or_create_collection(name = "Procom_Competitions", embedding_function = sentence_transformer_ef)

In [ ]:
for i, text in enumerate(document):
   client.add(documents = document[i], metadatas={"source": f"Procom_Docs_{i + 1}"}, ids=[str(i)])


In [ ]:
t = client.query(query_texts = "What is AI showdown about", n_results = 1)
t['documents'][0][0]

' AI Showdown \n Overview \n The  AI  Showdown  is  a  dynamic  competition  where  participants  delve  into  real-world  datasets, \n applying  their  expertise  to  solve  intricate  problems.  The  challenge  tests  skills  in  data  cleaning, \n exploratory  data  analysis,  insight  and  pattern  extraction,  and  predictive  modeling.  Participants \n are  evaluated  based  on  solution  accuracy ,  model  effectiveness,  and  their  ability  to  communicate \n insights.  The  competition  is  designed  to  cater  to  participants  of  all  levels,  whether  you  are  a \n newbie,  a  beginner ,  or  a  seasoned  ML  expert.  The  event  not  only  offers  winners  valuable  prizes \n but  also  recognition,  making  the  challenge  a  dynamic  platform  for  showcasing  and  advancing \n data science and machine learning expertise. \n Rounds \n The competition consists of two rounds. \n Round 1: Exploratory Data Analysis (EDA) and Modeling, T eams will be provided with a CSV \n

In [ ]:
def get_completion_from_messages(contents, User_input, temperature=0):
    messages =  [
        {'role':'system', 'content':"""You are a chatbot of an event called 'PROCOM', and when ever you reply, you will write 'PROCOM BOT responded with: ' and then you will give the response.
        if there is something question related in the provided data then give suitable response for the question that is relevant and the question that is not relevant, if you are unable to respond to the query, then kindly answer with irrelevant information asked. 
        """},    
        {'role':'user', 'content':contents},
        {'role':'assistant', 'content':User_input}
    ]                                 
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]

In [ ]:
user_input = str(input("Enter a query: "))
content = client.query(query_texts = user_input, n_results = 1)
content['documents'][0][0]
#response = get_completion_from_messages(content['documents'][0], user_input, temperature = 0.7)
#print(response)

' AI Showdown \n Overview \n The  AI  Showdown  is  a  dynamic  competition  where  participants  delve  into  real-world  datasets, \n applying  their  expertise  to  solve  intricate  problems.  The  challenge  tests  skills  in  data  cleaning, \n exploratory  data  analysis,  insight  and  pattern  extraction,  and  predictive  modeling.  Participants \n are  evaluated  based  on  solution  accuracy ,  model  effectiveness,  and  their  ability  to  communicate \n insights.  The  competition  is  designed  to  cater  to  participants  of  all  levels,  whether  you  are  a \n newbie,  a  beginner ,  or  a  seasoned  ML  expert.  The  event  not  only  offers  winners  valuable  prizes \n but  also  recognition,  making  the  challenge  a  dynamic  platform  for  showcasing  and  advancing \n data science and machine learning expertise. \n Rounds \n The competition consists of two rounds. \n Round 1: Exploratory Data Analysis (EDA) and Modeling, T eams will be provided with a CSV \n